In [1]:
%matplotlib inline
import pandas
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np


pandas.set_option('display.max_columns', None)  
pandas.set_option('display.expand_frame_repr', False)
#pandas.set_option('max_colwidth', -1)
pandas.set_option('display.precision', 3)

In [2]:
df = pandas.read_csv('data.csv', sep=',', na_values="")
print(df[0:10])

       Date      Time                                         Tweet_Text  Type Media_Type                    Hashtags   Tweet_Id                                          Tweet_Url  twt_favourites_IS_THIS_LIKE_QUESTION_MARK  Retweets  Unnamed: 10  Unnamed: 11
0  16-11-11  15:26:37  Today we express our deepest gratitude to all ...  text      photo                   ThankAVet  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     127213     41112          NaN          NaN
1  16-11-11  13:33:35  Busy day planned in New York. Will soon be mak...  text        NaN                         NaN  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     141527     28654          NaN          NaN
2  16-11-11  11:14:20  Love the fact that the small groups of protest...  text        NaN                         NaN  7.970e+17  https://twitter.com/realDonaldTrump/status/797...                                     183729 

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# Preparación de las herramientas de preprocesamiento de texto
nltk.download('stopwords')
stop = set(stopwords.words('english')) 
sno = SnowballStemmer('english') 

def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned

# Procesamiento del texto de los tweets
final_string = []
for sent in df['Tweet_Text'].values:
    filtered_sentence = []
    sent = cleanhtml(sent) 
    for w in sent.split():
        cleaned_words = cleanpunc(w)
        if cleaned_words.isalpha() and len(cleaned_words) > 2:
            if cleaned_words.lower() not in stop:
                stemmed_word = sno.stem(cleaned_words.lower())
                filtered_sentence.append(stemmed_word)
    final_string.append(" ".join(filtered_sentence))

# Añadiendo la columna de tweets limpios al DataFrame
df['cleaned_tweet'] = final_string

# Visualización de los primeros registros del DataFrame modificado
print(df["cleaned_tweet"].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marcp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    today express deepest gratitud serv arm thankavet
1    busi day plan new soon make import decis peopl...
2    love fact small group protest last night passi...
3      open success presidenti profession incit unfair
4    fantast day met presid obama first realli good...
Name: cleaned_tweet, dtype: object


In [4]:

media_likes= df["twt_favourites_IS_THIS_LIKE_QUESTION_MARK"].median()
media_retweets=df["Retweets"].median()
df['Viral'] = df.apply(lambda x: "mucho" if (x['twt_favourites_IS_THIS_LIKE_QUESTION_MARK'] + x['Retweets']) >= (media_likes + media_retweets) * 3
                       else ("moderado" if (x['twt_favourites_IS_THIS_LIKE_QUESTION_MARK'] + x['Retweets']) >= (media_likes + media_retweets)*0.5
                             else "poco"), axis=1)

In [5]:
print(df["Viral"].value_counts())

Viral
moderado    3464
poco        2272
mucho       1639
Name: count, dtype: int64


In [43]:
df.drop('Tweet_Text', axis=1, inplace=True)
df.drop('Tweet_Id', axis=1, inplace=True)
df.drop('Tweet_Url', axis=1, inplace=True)
df.drop('twt_favourites_IS_THIS_LIKE_QUESTION_MARK', axis=1, inplace=True)
df.drop('Retweets', axis=1, inplace=True)
df.drop('Unnamed: 10', axis=1, inplace=True)
df.drop('Unnamed: 11', axis=1, inplace=True)
y=df["Viral"].values
# Definir X como todas las columnas excepto 'Viral'
X= df.drop('Viral', axis=1).values


KeyError: "['Tweet_Text'] not found in axis"

In [7]:
print(df[0:10])

       Date      Time  Type Media_Type                    Hashtags                                      cleaned_tweet  Viral
0  16-11-11  15:26:37  text      photo                   ThankAVet  today express deepest gratitud serv arm thankavet  mucho
1  16-11-11  13:33:35  text        NaN                         NaN  busi day plan new soon make import decis peopl...  mucho
2  16-11-11  11:14:20  text        NaN                         NaN  love fact small group protest last night passi...  mucho
3  16-11-11   2:19:44  text        NaN                         NaN    open success presidenti profession incit unfair  mucho
4  16-11-11   2:10:46  text        NaN                         NaN  fantast day met presid obama first realli good...  mucho
5  16-11-10  19:31:27  text      photo                         NaN             happi birthday marin corp thank servic  mucho
6  16-11-09  11:36:58  text        NaN                         NaN  beauti import even forgotten man woman never f...  mucho


## 3- Simple cross-validation

In [9]:
import sklearn.model_selection as cv 
(X_train, X_test,  y_train, y_test) = cv.train_test_split(X, y, test_size=.3, random_state=1)
print('Size training: ',X_train.shape)
print('Size test: ',X_test.shape)
print(X_test[:5])

Size training:  (5162, 6)
Size test:  (2213, 6)
[['16-04-25' '18:11:08' 'link' nan nan 'dead clinton trump']
 ['15-11-13' '20:18:12' 'text' nan nan
  'said four week ago putinnev said green separ piecesgreat rate']
 ['16-02-17' '3:04:22' 'text' nan nan
  'explain know beat hillari win state']
 ['16-01-15' '15:30:38' 'text' nan nan
  'lindsey graham embarrass fail run presid embarrass endors']
 ['16-02-16' '11:22:02' 'text' nan nan
  'new ppp poll trump rubio bush debat even stack rnc wonder']]


In [13]:
print( X_train[:, -1])

['great honor result cnn poll believ final result even better'
 'drug pour ice endors debat bigleaguetruth'
 'michigan gop poll trump rubio cruz kasich carson wow' ...
 'peopl tweet million low poll big hit ad'
 'thank great crowd support get vote let maga'
 'saw share want love hero cute']


In [19]:
## Conversion from text to Bag of Words representation as a table

from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
Bow_train = count_vectorizer.fit_transform( X_train[:, -1])
Bow_test = count_vectorizer.transform( X_test[:, -1])

feature_names = count_vectorizer.get_feature_names_out()


print('Shape of table for training: ',Bow_train.shape)
print('Shape of table for test: ',Bow_test.shape)
print( Bow_train[:10])

Shape of table for training:  (5162, 4108)
Shape of table for test:  (2213, 4108)
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 84 stored elements and shape (10, 4108)>
  Coords	Values
  (0, 1567)	1
  (0, 1710)	1
  (0, 2986)	2
  (0, 674)	1
  (0, 2719)	1
  (0, 317)	1
  (0, 1356)	1
  (0, 1232)	1
  (0, 345)	1
  (1, 1099)	1
  (1, 2739)	1
  (1, 1754)	1
  (1, 1181)	1
  (1, 899)	1
  (1, 354)	1
  (2, 2719)	1
  (2, 2271)	1
  (2, 1532)	1
  (2, 3727)	1
  (2, 3065)	1
  (2, 854)	1
  (2, 1962)	1
  (2, 547)	1
  (2, 4058)	1
  (3, 3065)	1
  :	:
  (7, 490)	1
  (7, 2104)	1
  (7, 481)	1
  (7, 340)	1
  (7, 2893)	1
  (7, 1224)	1
  (7, 211)	1
  (7, 113)	1
  (8, 2634)	1
  (8, 2053)	1
  (8, 2057)	1
  (8, 380)	1
  (8, 351)	1
  (8, 3650)	1
  (9, 1567)	1
  (9, 2719)	1
  (9, 3727)	1
  (9, 113)	1
  (9, 510)	1
  (9, 2470)	1
  (9, 3947)	1
  (9, 2174)	1
  (9, 118)	1
  (9, 2707)	1
  (9, 2045)	1


In [25]:
df_bow_train = pandas.DataFrame(Bow_train.toarray(), columns=feature_names)
df_bow_train.to_csv('bow_train.csv', index=False)

In [38]:
import sklearn                         # Llibreia de DM
import sklearn.datasets as ds            # Per carregar mÃ©s facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb 
knc = nb.KNeighborsClassifier()

# Train the classifier
knc.fit(Bow_train, y_train)

# Obtain accuracy score of learned classifier on test data
print(knc.score(Bow_test, y_test))


0.47401717126073206


In [39]:
# More4 information with confussion matrix
from sklearn.metrics import confusion_matrix

y_pred = knc.predict(Bow_test)
print(sklearn.metrics.confusion_matrix(y_test, y_pred))

[[630  46 380]
 [273  64 147]
 [302  16 355]]


In [40]:
# Obtain Recall, Precision and F-Measure for each class
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    moderado       0.52      0.60      0.56      1056
       mucho       0.51      0.13      0.21       484
        poco       0.40      0.53      0.46       673

    accuracy                           0.47      2213
   macro avg       0.48      0.42      0.41      2213
weighted avg       0.48      0.47      0.45      2213

